In [1]:
import numpy as np
import pandas as po
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
df = po.read_csv('data/EnergyConsumption_Theft.csv')
df = df.sample(frac=1).reset_index(drop = True) # shuffle df so that classes are evenly distributed in train-val-test

In [3]:
df = df.drop(['CONS_NO'], axis = 1)

In [4]:
df.isnull().sum().sum()

0

In [5]:
df

,FLAG,2014/1/1,2014/1/10,2014/1/11,2014/1/12,2014/1/13,2014/1/14,2014/1/15,2014/1/16,2014/1/17,...,2016/9/28,2016/9/29,2016/9/3,2016/9/30,2016/9/4,2016/9/5,2016/9/6,2016/9/7,2016/9/8,2016/9/9
0,0,6.12,7.44,9.20,8.56,9.44,7.52,6.24,9.27,10.18,...,11.340000,5.230000,7.260000,6.670000,5.230000,8.270000,7.530000,7.730000,5.140000,6.960000
1,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,3.800000,3.250000,2.230000,2.540000,2.210000,0.360000,0.570000,2.670000
2,0,9.05,0.00,7.02,4.65,3.28,3.99,5.43,0.00,5.00,...,10.070000,13.360000,22.530000,9.660000,25.550000,18.280000,16.600000,14.490000,11.140000,8.660000
3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,5.454006,5.454006,5.454006,5.454006,4.550000,5.454006,5.454006,5.454006,5.454006,5.454006
4,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.173928,0.050000,0.060000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42367,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,12.020000,10.160000,13.350000,10.940000,10.300000,11.440000,10.200000,11.780000,10.810000,11.130000
42368,0,2.71,2.08,1.85,2.07,2.32,0.00,1.77,1.61,2.81,...,5.060124,3.910000,5.060124,4.510000,5.060124,4.590000,4.550000,3.930000,4.370000,5.060124
42369,0,9.92,14.44,8.71,10.57,9.62,0.00,0.00,11.87,10.36,...,12.510000,8.350000,18.870000,10.750000,14.460000,13.650000,14.670000,12.220000,12.970000,9.940000
42370,0,10.42,10.99,6.57,6.12,8.65,8.43,9.23,10.21,9.24,...,21.410000,20.970000,8.410000,17.070000,16.570000,24.600000,25.565727,25.565727,13.390000,18.700000


In [6]:
zeros_df = df[df==0]
num_zeros = []
for i in tqdm(range(len(zeros_df))):
    try:
        num_zeros.append(zeros_df.iloc[i].value_counts()[0])
    except KeyError:
        num_zeros.append(0)

100%|██████████| 42372/42372 [00:33<00:00, 1266.10it/s]


In [7]:
df['num_zeros'] = num_zeros

In [8]:
#df = df.drop(sorted(range(len(num_zeros)), key=lambda i: num_zeros[i], reverse=True)[:len(num_zeros) - 25000], axis = 0).reset_index(drop = True)

In [10]:
df['FLAG'].value_counts()

0    38757
1     3615
Name: FLAG, dtype: int64

In [11]:
df_0 = df[df['FLAG'] == 0].sort_values('num_zeros')[:3615].reset_index(drop=True)
df_1 = df[df['FLAG'] == 1].reset_index(drop=True)

In [12]:
df_f = po.concat([df_0, df_1], axis=0, ignore_index=True).reset_index(drop=True)

In [47]:
df_f = df_f.sample(frac=1).reset_index(drop = True)

In [48]:
df_f['FLAG'].value_counts()

1    3615
0    3615
Name: FLAG, dtype: int64

In [49]:
X = df_f.drop(['FLAG'], axis = 1)
y = po.get_dummies(df_f['FLAG'])

In [50]:
X_train = X[:5000].values.reshape(-1, len(X.columns), 1)
X_val = X[5000:6000].values.reshape(-1, len(X.columns), 1)
X_test = X[6000:].values.reshape(-1, len(X.columns), 1)

In [51]:
y_train = y[:5000].values.reshape(-1, 2)
y_val = y[5000:6000].values.reshape(-1, 2)
y_test = y[6000:].values.reshape(-1, 2)

In [52]:
with open('data/train_splits_undersampled.npz', 'wb') as save_file:
    np.savez(save_file, X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train, y_val=y_val, y_test=y_test)

In [54]:
with open('data/train_splits_undersampled.npz', 'rb') as load_file:
    npzfile = np.load(load_file)    
    
    X_train = npzfile['X_train']
    X_val = npzfile['X_val']
    X_test = npzfile['X_test']
    
    y_train = npzfile['y_train']
    y_val = npzfile['y_val']
    y_test = npzfile['y_test']

In [55]:
y_train.shape

(5000, 2)

In [56]:
X_train.shape

(5000, 1035, 1)

In [57]:
X_val.shape

(1000, 1035, 1)

In [58]:
X_test.shape

(1230, 1035, 1)

In [59]:
y_val.shape

(1000, 2)